# Bayesian Network Example: Disease, Fever, Cough


This notebook demonstrates how to **build and use a Bayesian Network** using the **pgmpy** library in Python.

We will model a simple **medical diagnosis problem**:

- Variables: `Disease`, `Fever`, `Cough`
- Dependencies: `Disease → Fever`, `Disease → Cough`
- Goal: Use **Bayesian Inference** to compute probabilities given evidence.


In [ ]:
!pip install pgmpy

In [ ]:

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination


## Step 1 & 2: Define the Bayesian Network structure

In [ ]:

# Structure: Disease → Fever, Disease → Cough
model = BayesianNetwork([('Disease', 'Fever'),
                         ('Disease', 'Cough')])
print("Model structure created successfully!")


## Step 3: Define Conditional Probability Distributions (CPDs)

In [ ]:

# Prior probability for Disease
cpd_disease = TabularCPD(variable='Disease',
                         variable_card=2,  # 2 states: No(0), Yes(1)
                         values=[[0.99],   # P(Disease=No)
                                 [0.01]])  # P(Disease=Yes)

# Fever depends on Disease
cpd_fever = TabularCPD(variable='Fever',
                       variable_card=2,
                       values=[[0.2, 0.1],   # P(Fever=No | Disease=No/Yes)
                               [0.8, 0.9]],  # P(Fever=Yes | Disease=No/Yes)
                       evidence=['Disease'],
                       evidence_card=[2])

# Cough depends on Disease
cpd_cough = TabularCPD(variable='Cough',
                       variable_card=2,
                       values=[[0.3, 0.2],   # P(Cough=No | Disease=No/Yes)
                               [0.7, 0.8]],  # P(Cough=Yes | Disease=No/Yes)
                       evidence=['Disease'],
                       evidence_card=[2])

# Add CPDs to model
model.add_cpds(cpd_disease, cpd_fever, cpd_cough)
print("CPDs added successfully!")


## Step 4: Verify the Model

In [ ]:
print('Model check:', model.check_model())

## Step 5: Perform Inference

In [ ]:

inference = VariableElimination(model)

# Predictive Inference: P(Fever | Disease=Yes)
print("P(Fever | Disease=Yes):")
print(inference.query(variables=['Fever'], evidence={'Disease': 1}))

# Diagnostic Inference: P(Disease | Fever=Yes, Cough=Yes)
print("\nP(Disease | Fever=Yes, Cough=Yes):")
print(inference.query(variables=['Disease'], evidence={'Fever': 1, 'Cough': 1}))
